In [ ]:
!pip install biopython

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

def filter_faa_sequences(input_faa, output_kept_faa, output_removed_faa,
                         ambiguous_threshold=0.05, min_length=50, filter_rare=True):
    """
    Filter protist sequences in a .faa FASTA file based on:
    - Ambiguous amino acids (X, B, Z)
    - Sequence length
    - Rare amino acids (U, O)
    - Duplicate sequences (exact sequence match)

    Args:
        input_faa (str): Path to input .faa file
        output_kept_faa (str): Path to output kept sequences .faa file
        output_removed_faa (str): Path to output removed sequences .faa file
        ambiguous_threshold (float): Max allowed proportion of ambiguous residues (default: 0.05)
        min_length (int): Minimum sequence length (default: 50)
    """
    ambiguous_residues = set('XBZ')
    rare_residues = set('UO')

    records = list(SeqIO.parse(input_faa, "fasta"))
    kept = []
    removed = []
    seen_sequences = {}

    for record in records:
        seq = str(record.seq).upper()
        length = len(seq)
        reason = None

        # Check for duplicates
        if seq in seen_sequences:
            reason = f"Duplicate of {seen_sequences[seq]}"
        else:
            seen_sequences[seq] = record.id

            # Length check
            if length < min_length:
                reason = f"Too short (<{min_length} aa)"
            else:
                # Ambiguous residue check
                num_ambiguous = sum(seq.count(res) for res in ambiguous_residues)
                frac_ambiguous = num_ambiguous / length
                if frac_ambiguous > ambiguous_threshold:
                    reason = f"Ambiguous residues > {ambiguous_threshold*100:.1f}%"
                else:
                    # Optional: Rare amino acids check
                    if filter_rare and any(res in seq for res in rare_residues):
                        reason = f"Rare amino acid (U/O) present"

        if reason:
            new_record = SeqRecord(Seq(seq), id=record.id, description=reason)
            removed.append(new_record)
        else:
            kept.append(record)

    # Write kept and removed sequences to separate files
    SeqIO.write(kept, output_kept_faa, "fasta")
    SeqIO.write(removed, output_removed_faa, "fasta")

    # Summary report
    print(f"Filtering complete: {len(removed)} sequences removed, {len(kept)} sequences kept out of {len(records)} total.")
    print(f"Kept sequences saved to: {output_kept_faa}")
    print(f"Removed sequences saved to: {output_removed_faa}")

    return kept, removed


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.3 MB/s eta 0:00:00
